# Design Patterns

## Creational Patterns
These design patterns concern creating objects.

### Factory Method

We may not always know what kind of objects we want to create in advance.
Some objects can be created only at execution time after a user requests so.
<br>

__Example(s)__: 
- A user may click on a certain button that creates an object.
- A user may create documents of different types.

<br>
__Implementation__:
Create a function, namely the factory, that takes an input __string__ and outputs an __object__. 
Factory is a _static_ method, i.e., it is bound to a class rather than its object; hence, it doesn't require a class instance creation.

In [0]:
class Document:
    def factory(type):
        if type == "PdfDocument": 
            return PdfDoc()
        if type == "WordDocument": 
            return WordDoc()
        raise AssertionError("Bad document creation: " + type)
        
    # staticmethod() converts the given function to a static method
    # you can use the Python decorator @staticmethod 
    # which will be covered later    
    factory = staticmethod(factory)


# Subclasses of Document
class PdfDoc(Document):
    def show(self): 
        print("Showing a PDF...")

        
class WordDoc(Document):
    def show(self): 
        print("Showing a MS Word...")

        
if __name__ == "__main__":  
    # Create object using the factory method.
    # Calling a static method does not require creating an instance
    firstdoc = Document.factory("PdfDocument")
    firstdoc.show()

    seconddoc = Document.factory("WordDocument")
    seconddoc.show()

    thirddoc = Document.factory("RtfDocument")

Showing a PDF...
Showing a MS Word...


### Singleton
The Singleton pattern is used when we want to guarantee that only one instance of a given class exists during runtime.
<br>

__Examples__:
- Controlling concurrent access to a shared resource
- Managing database connections
- Printer spooler
- Logging class

__Implementation__: Modify the constructor to store the first instance of a class in a class attribute. 
Return the same attribute every time that the constructor is called.

In [0]:
class Logger:
    def __new__(cls):
        if not hasattr(cls, '_logger'):
            cls._logger = super().__new__(cls)
        return cls._logger


if __name__ == '__main__':
    objA = Logger()
    objB = Logger()
    print('objA id: {0}'.format(id(objA)))
    print('objB id: {0}'.format(id(objB)))

objA id: 140661925795264
objB id: 140661925795264


__Exercise__: Create a Singleton class, `OnlyK`, which can create to `K` instances.
Assume the objects are database connections and you only have a license to use a fixed quantity of these at any one time.

In [0]:
K = 3

class OnlyK(object):
    '''Singleton class which manages a fixed number of its own objects'''
    __instance_count = 0
    
    def __new__(cls):
        if cls.__instance_count < K:
            cls.__instance_count += 1
            cls.__last_instance = super().__new__(cls)
        return cls.__last_instance
  

# Test your implementation
if __name__ == '__main__':
    first = OnlyK()
    print('obj id: {0}'.format(id(first)))
    second = OnlyK()
    print('obj id: {0}'.format(id(second)))
    third = OnlyK()
    print('obj id: {0}'.format(id(third)))
    fourth = OnlyK()
    print('obj id: {0}'.format(id(fourth)))

obj id: 139760682379416
obj id: 139760682377792
obj id: 139760682456120
obj id: 139760682456120


### Monostate

The Monostate pattern is a way to implement singleton behavior, 
but instead of having only one instance of a class, 
there are multiple instances that share the same state. 
In other words, the focus is on sharing state instead of sharing instance identity.
<br>

__Examples__:
- Managing database connections
- Printer spooler
- Logging class
<br>

__Implementation__:
In Python, each instance has its own dictionary. 
The Monostate pattern modifies this so that all instances have the same dictionary.
In this example, the `__shared_state` attribute will be the dictionary
shared between all instances, and this is ensured by assigining
`__shared_state` to the `__dict__` variable when initializing a new
instance (i.e., in the `__init__` method). Other attributes are usually
added to the instance's attribute dictionary, but, since the attribute
dictionary itself is shared (which is `__shared_state`), all other
attributes will also be shared.

In [20]:
class Borg:
    # this dictionary is a class attribute
    __shared_state = {}

    def __init__(self):
        # ensure that all instances have the same state (i.e., attributes)
        self.__dict__ = self.__shared_state
        
        self.state = 'Init'
        


class YourBorg(Borg):
    pass


if __name__ == '__main__':
    rm1 = Borg()
    rm2 = Borg()

    rm1.state = 'Idle'
    
    print('rm1 dictionary: {0}'.format(rm1.__dict__))
    print('rm2 dictionary: {0}'.format(rm2.__dict__))
    
    
    rm2.state = 'Running'

    print('rm1 state: {0}'.format(rm1.state))
    print('rm2 state: {0}'.format(rm2.state))
    
    print('rm1 dictionary: {0}'.format(rm1.__dict__))
    print('rm2 dictionary: {0}'.format(rm2.__dict__))

    rm2.state = 'Halt'

    print('rm1 state: {0}'.format(rm1.state))
    print('rm2 state: {0}'.format(rm2.state))
    
    print('rm1 dictionary: {0}'.format(rm1.__dict__))
    print('rm2 dictionary: {0}'.format(rm2.__dict__))

    
    rm3 = YourBorg()

    print('rm1 state: {0}'.format(rm1.state))
    print('rm2 state: {0}'.format(rm2.state))
    print('rm3 state: {0}'.format(rm3.state))
    
    print('rm1 dictionary: {0}'.format(rm1.__dict__))
    print('rm2 dictionary: {0}'.format(rm2.__dict__))
    print('rm3 dictionary: {0}'.format(rm3.__dict__))
    
    # Note the difference with Singleton
    print('rm1 id: {0}'.format(id(rm1)))
    print('rm2 id: {0}'.format(id(rm2)))
    print('rm3 id: {0}'.format(id(rm3)))

rm1 dictionary: {'state': 'Idle'}
rm2 dictionary: {'state': 'Idle'}
rm1 state: Running
rm2 state: Running
rm1 dictionary: {'state': 'Running'}
rm2 dictionary: {'state': 'Running'}
rm1 state: Halt
rm2 state: Halt
rm1 dictionary: {'state': 'Halt'}
rm2 dictionary: {'state': 'Halt'}
rm1 state: Init
rm2 state: Init
rm3 state: Init
rm1 dictionary: {'state': 'Init'}
rm2 dictionary: {'state': 'Init'}
rm3 dictionary: {'state': 'Init'}
rm1 id: 140594264590992
rm2 id: 140594264933880
rm3 id: 140594264494824


### Builder

The Builder pattern decouples the creation of a complex object and its representation,
so that the same process can be reused to build objects from the same
family.
<br>

__Implementation__: This particular example uses a director function to abtract the construction of a building. The user specifies a Builder (House or Flat) and the director specifies the methods in the order necessary creating a different building depending on the specified specification (through the Builder class)

In [27]:
def construct_building(builder):
    builder.new_building()
    # different stages of the build
    builder.build_frame()
    builder.build_floor()
    builder.build_walls()
    
    return builder.building


# Abstract Builder
class Builder(object):
  
    def __init__(self):
        self.building = None

    def new_building(self):
        self.building = Building()
        
    def build_frame(self):
        raise NotImplementedError

    def build_floor(self):
        raise NotImplementedError

    def build_walls(self):
        raise NotImplementedError


# Concrete Builder Classes
class BuilderHouse(Builder):

    def build_floor(self):
        self.building.floor = 'One'

    def build_walls(self):
        self.building.walls = 'Brick'
        
    def build_frame(self):
        self.building.frame = 'Wood'


class BuilderCondo(Builder):

    def build_floor(self):
        self.building.floor = 'Five'

    def build_walls(self):
        self.building.walls = 'Concrete'
        
    def build_frame(self):
        self.building.frame = 'Steel'


# Product
class Building(object):
    def __init__(self):
        self.floor = None
        self.walls = None
        self.frame = None

    def __str__(self):
        return 'Floor: {0.floor}|Walls: {0.walls}|Frame: {0.frame}'.format(self)


# Client
if __name__ == "__main__":
    building = construct_building(BuilderHouse())
    print(building)
    building = construct_building(BuilderCondo())
    print(building)

Floor: One|Walls: Brick|Frame: Wood
Floor: Five|Walls: Concrete|Frame: Steel


### Prototype

Create new object instances by cloning an existing object.
<br>

__Example(s)__: 
- When you have a very complex object that cost too much to instantiate

__Implementation__:

In [34]:
from copy import deepcopy

class Prototype:
    def __init__(self):
        self._objs = {}
        
    def registerObject(self, name, obj):
        """
        register an object.
        """
        self._objs[name] = obj
        
    def unregisterObject(self, name):
        """unregister an object"""
        del self._objs[name]
        
    def clone(self, name, **attr):
        """clone a registered object and add/replace attr"""
        
        # **-operator can be used for unpacking key, 
        # value pairs out of a dictionary
        
        # A deep copy constructs a new compound object and then, 
        # recursively, inserts copies (not references) into it 
        # of the objects found in the original
        obj = deepcopy(self._objs[name])
        obj.__dict__.update(attr)
        return obj

      
class A:
    def __init__(self, value='default', b_obj=None):
        self.value = value
        self.obj = b_obj
        

class B:
    '''Some dummy class'''
    pass

  
if __name__ == '__main__':
    prototype = Prototype()
  
    a = A('new', B())
    prototype.registerObject("a", a)
    print(a.__dict__)
    
    b = prototype.clone("a", new_var=1)
    prototype.registerObject("b", b)
    print(b.__dict__)
    
    c = prototype.clone("b", value='changed', new_var=5)
    print(c.__dict__)

{'value': 'new', 'obj': <__main__.B object at 0x7fdea733cc88>}
{'value': 'new', 'obj': <__main__.B object at 0x7fdea733ca58>, 'new_var': 1}
{'value': 'changed', 'obj': <__main__.B object at 0x7fdea733ca90>, 'new_var': 5}


## Structural Patterns

These design patterns concern class and object composition.

### Facade
The Facade pattern is used to expose a controlled, well thought-out subset of functionality.
<br>

__Examples__:
- In `os.path` module: `basename`, `dirname` functions are facades for splitting + indexing: <br>
`os.path.dirname('/etc/ssh/ssh_config') is '/etc/ssh'
os.path.basename('/etc/ssh/ssh_config') is 'ssh_config'`
<br>

__Implementation__:

In [52]:
class Tyre:
    def __init__(self, name, size=225, pressure=25):
        self.name = name
        self.pressure = pressure
        self.size = size
        
    def inflate(self, psi=30):
        self.pressure = psi # psi


class Tank:
    def __init__(self, level):
        self.level = level
        
    def fillup(self, level=100):
        self.level = level
        
class Car:
    def __init__(self):
        self._tyres = [Tyre('front_left'), Tyre('front_right'), \
                       Tyre('rear_left'), Tyre('rear_right')]
        self._tank = Tank(70)

    def tyres_pressure(self):
        return [str(tyre.pressure) + ' psi' for tyre in self._tyres]

    def fuel_level(self):
        return str(45 * self._tank.level / 100) + ' litres'
      
      
if __name__ == "__main__":
    mycar = Car()
    print(mycar.tyres_pressure())
    print(mycar.fuel_level())

['25 psi', '25 psi', '25 psi', '25 psi']
31.5 litres


### Proxy
The Proxy pattern provides a placeholder for another object to control access to it, shielding the object from its clients. There may be a number of reasons for this: reference counting, different levels of access rights, and lazy evaluation of the state of the object.
<br>

__Example(s)__:
- When you don’t want the client programmer to have full access to the proxied object.
- To add additional actions when the proxied object is accessed. For example, or to keep track of the number of references that are held for a particular object
<br>

__Implementation__: Using `__getattr__` method which is a built-in delegation mechanism 

In [0]:
class RGB:
    def __init__(self, red, green, blue):
        self.__red = red
        self.__green = green
        self.__blue = blue
    def Red( self ):
        return self.__red
    def Green( self ):
        return self.__green
    def Blue( self ):
        return self.__blue
      
      
class Proxy:
    def __init__(self, shielded_obj):
        self.__shielded = shielded_obj
        self.__access_count = 0
        
    def __getattr__(self, name):
        if hasattr(self.__shielded, name):
            self.__access_count += 1
            return getattr(self.__shielded, name)
        else:
            raise NameError
          
    def count_access(self):
        print(self.__access_count)
          
          
if __name__ == "__main__":         
    rgb = RGB(100, 192, 240)
    rgb.Red()

    proxy = Proxy(rgb)
    proxy.Green()
    proxy.count_access()
    proxy.Red()
    proxy.count_access()
    proxy.Green()
    proxy.count_access()

1
2
3


### Adapter

The Adapter pattern provides a different interface for a class; thus,
it is useful for integrating classes that couldn't be
integrated otherwise due to their incompatible interfaces.
<br>

__Implementation__: Wrap the interface of a class around that of an already existing class.

In [61]:
class Dog:

    def __init__(self):
        self.name = "Dog"

    def bark(self):
        return "woof!"


class Cat:

    def __init__(self):
        self.name = "Cat"

    def meow(self):
        return "meow!"


class Human:

    def __init__(self):
        self.name = "Human"

    def speak(self):
        return "'hello'"


class Car:

    def __init__(self):
        self.name = "Car"

    def make_noise(self, octane_level):
        return "vroom{0}".format("!" * octane_level)


class Adapter:

    def __init__(self, obj, **adapted_methods):
        '''We set the adapted methods in the object's dict'''
        self.obj = obj
        self.__dict__.update(adapted_methods)

    def __getattr__(self, attr):
        '''All non-adapted calls are passed to the object'''
        return getattr(self.obj, attr)

    def original_dict(self):
        '''Print original object dict'''
        return self.obj.__dict__


      
if __name__ == "__main__": 
    # objects with incompatible interfaces
    dog = Dog()
    cat = Cat()
    human = Human()
    car = Car()
    
    # list of objects with wrapped interfaces
    objects = []
    
    # Adapter provides the new interface
    objects.append(Adapter(dog, make_noise=dog.bark))
    objects.append(Adapter(cat, make_noise=cat.meow))
    objects.append(Adapter(human, make_noise=human.speak))
    objects.append(Adapter(car, make_noise=lambda: car.make_noise(3)))
    
    print(objects[0].__dict__)
    print(objects[0].original_dict())
    
    # User requires an interface which has a method called make_noise()
    # These objects do not have the right interface so we must use an adapter
    for obj in objects:
        print("A {0} goes {1}".format(obj.name, obj.make_noise()))

{'obj': <__main__.Dog object at 0x7fdea72d4048>, 'make_noise': <bound method Dog.bark of <__main__.Dog object at 0x7fdea72d4048>>}
{'name': 'Dog'}
A Dog goes woof!
A Cat goes meow!
A Human goes 'hello'
A Car goes vroom!!!


#### Adapter vs. Facade
- Adapter is about supplying a given protocol required by client-code
- Facade is about simplifying a rich interface when just a subset is often needed

### Decorator
The Decorator pattern is used to dynamically add a new feature to an
object without changing its implementation. It differs from
inheritance because the new feature is added only to that particular
object, not to the entire subclass.
It's used when subclassing creates too many classes.
<br>

__Example(s)__:
- Add functionalities to methods, like permissions or subscription to an event
<br>

__Implementation__: Using inheritance

In [0]:
class TextTag:
    '''Represents a base text tag'''
    def __init__(self, text):
        self._text = text

    def render(self):
        return self._text


class BoldWrapper(TextTag):
    '''Wraps a tag in <b>'''
    def __init__(self, wrapped):
        self._wrapped = wrapped

    def render(self):
        return "<b>{}</b>".format(self._wrapped.render())


class ItalicWrapper(TextTag):
    '''Wraps a tag in <i>'''
    def __init__(self, wrapped):
        self._wrapped = wrapped

    def render(self):
        return "<i>{}</i>".format(self._wrapped.render())

      
if __name__ == '__main__':
    simple_msg = TextTag("hello, world!")
    formatted_msg = ItalicWrapper(BoldWrapper(simple_msg))
    print("before:", simple_msg.render())
    print("after:", formatted_msg.render())

before: hello, world!
after: <i><b>hello, world!</b></i>


## Behavioral Patterns
These design patterns are specifically concerned with communication between objects.

### Chain of Responsibility

The Chain of Responsibility pattern is useful to create a system that can serve different requests in a hierarchical manner.
<br>

__Example(s)__:
- Writing event handlers for the graphical user interface
<br>

__Implementation__: Instead of calling a single method to satisfy a request, multiple methods in the chain have a chance to satisfy the request, so it has the flavor of an expert system. Since the chain is effectively a linked list, it can be dynamically created, so you could also think of it as a more general, dynamically-built switch statement.

In [62]:
class Event:
    def __init__(self, name):
        self.name = name


# Abstract Class        
class Widget:
    def __init__(self, parent = None):
        # Create the chain of responsibility
        self.__parent = parent
        
    def Handle(self, event):
        handler = 'Handle_' + event.name
        if hasattr(self, handler):
            # Respond to the event if possible
            method = getattr(self, handler)
            method(event)
        elif self.__parent:
            # Use the chain of responsibility to ensure a response
            self.__parent.Handle(event)
        elif hasattr( self, 'HandleDefault' ):
            # If reached the end of the chain
            self.HandleDefault(event)
            
    def HandleDefault():
        raise NotImplementedError
            

class MainWindow(Widget):
    def Handle_close(self, event):
        print('MainWindow:', event.name)
        
    def HandleDefault(self, event):
        print('Default:', event.name)

        
class SendDialog(Widget):
    def Handle_click(self, event):
        print('SendDialog:', event.name)
        
    def HandleDefault(self, event):
        print('Default:', event.name)

        
class MsgText(Widget):
    def Handle_highlight(self, event):
        print('MsgText:', event.name)
        
    def HandleDefault(self, event):
        print('Default:', event.name)
        
        
# Our GUI is a simple window
mw = MainWindow()
# The window contains a dialog box
sd = SendDialog(mw)
# The dialog box has some text message
msg = MsgText(sd)

eclose = Event('close')
msg.Handle(eclose)

eclick = Event('click')
msg.Handle(eclick)

ehighlight = Event('highlight')
msg.Handle(ehighlight)

edown = Event('down')
msg.Handle(edown)

MainWindow: close
SendDialog: click
MsgText: highlight
Default: down


### Command

The Command pattern allows for choosing the operation at runtime.
<br>

__Example(s)__:
- Writing a shell that can execute commands and has a command history.
<br>

__Implementation__: Wrap a method that specifies the operation in an object which is then passed it to other methods or objects as a parameter. They can perform this particular operation at runtime, thereby creating new behaviour dynamically.

__Note__: Recall that functions are objects in Python. Thus, the Command pattern is often redundant.

In [0]:
import os


class Command:
    def execute(self): 
        raise NotImplementedError


class RenameFileCommand(Command):
    def __init__(self, from_name, to_name):
        self._from = from_name
        self._to = to_name

    def execute(self):
        os.rename(self._from, self._to)
        
        
class NewFileCommand(Command):
    def __init__(self, path):
        self._path = path

    def execute(self):
        basedir = os.path.dirname(self._path)
        if not os.path.exists(self._path):
            if not os.path.exists(basedir):
                os.makedirs(basedir)
            os.mknod(self._path)
        else:
            # Suppose this command always creates a new file
            os.remove(self._path)
            os.mknod(self._path)


class History:
    def __init__(self):
        self._commands = []

    def add(self, command):
        # Add a new command to the list of commands
        self._commands.append(command)
        
    def execute(self):
        # Execute existing commands one by one
        for command in self._commands:
            command.execute()


history = History()

history.add(NewFileCommand('logs/new.log'))
history.add(RenameFileCommand('logs/new.log', 'logs/old.log'))
history.add(RenameFileCommand('logs/old.log', 'logs/new.log'))

history.execute()

__Exercise__: Implement the undo method of `History`, `RenameFileCommand`, and `NewFileCommand`

In [0]:
import os

class Command:
    def execute(self): 
        pass
    
    def undo(self): 
        pass


class RenameFileCommand(Command):
    def __init__(self, from_name, to_name):
        self._from = from_name
        self._to = to_name

    def execute(self):
        os.rename(self._from, self._to)

    def undo(self):
        os.rename(self._to, self._from)
        
        
class NewFileCommand(Command):
    def __init__(self, path_name):
        self._path = path_name

    def execute(self):
        basedir = os.path.dirname(self._path)
        if not os.path.exists(self._path):
            if not os.path.exists(basedir):
                os.makedirs(basedir)
            os.mknod(self._path)
        else:
            # Suppose this command always creates a new file
            os.remove(self._path)
            os.mknod(self._path)

    def undo(self):
        os.remove(self._path)


class History:
    def __init__(self):
        self._commands = list()

    def add(self, command):
        self._commands.append(command)
        command.execute()

    def undo(self):
        self._commands.pop().undo()
        
#     def execute(self):
#         for command in self._commands:
#             command.execute()
            
            
history = History()
history.add(NewFileCommand('logs/new.log'))
history.add(RenameFileCommand('logs/new.log', 'logs/old.log'))
history.add(RenameFileCommand('logs/old.log', 'logs/new.log'))

# history.execute()

history.undo()
history.undo()
history.undo()

In [0]:
os.remove('logs/new.log')

### Strategy

The Strategy pattern allows for choosing the algorithm on-the-fly at runtime.
<br>

__Example(s)__:
- `sort()` takes a second optional argument that acts as a comparator object; this is a strategy.
<br>

__Implementation__: 

In [9]:
class MathFunc:
    pass


class FindMinima:
    def algorithm(self, func):
        pass


# The various strategies:
class QuasiNewtonMethod(FindMinima):
    def algorithm(self, func):
        return "Quasi-Newton method is being used..."

      
class NewtonsMethod(FindMinima):
    def algorithm(self, func):
        return "Newton's method is being used..."


class GradientDescentMethod(FindMinima):
    def algorithm(self, func):
        return "Gradient Descent method is being used..."


# The "Context" controls the strategy:
class OptoSolver:
    def __init__(self, strategy):
        self.strategy = strategy

    def minima(self, func):
        return self.strategy.algorithm(func)

    def change_algorithm(self, newAlgorithm):
        self.strategy = newAlgorithm



myfunc = MathFunc()
solver = OptoSolver(GradientDescentMethod())
print(solver.minima(myfunc))
solver.change_algorithm(NewtonsMethod())
print(solver.minima(myfunc))

Gradient Descent method is being used...
Newton's method is being used...


### Observer
The Observer pattern maintains a list of dependents and notifies them of any state changes. 
The Observer pattern describes a publish-subscribe relationship. The subscribers subscribe with a publisher for updates regarding that topic. The publisher, when triggered, publishes the topic updates to the right subscribers.
<br>

__Example(s)__:
- A server pushing updates to clients
- Event handlers subscribing to an event

__Implementation__: Observer is an *interface* class that only has one member function, `update( )`. This function is called by the object that’s being observed, when that object decides its time to update all its observers. 

In [16]:
class Observable:
    def __init__(self):
        self.observers = []
 
    def register(self, observer):
        # Register a new observer
        if not observer in self.observers:
            self.observers.append(observer)
 
    def unregister(self, observer):
        # Unregister an observer
        if observer in self.observers:
            self.observers.remove(observer)
 
    def unregister_all(self):
        # Unregister all observers
        if self.observers:
            del self.observers[:]
 
    def update_observers(self, *args, **kwargs):
        # Notify all registered observers
        for observer in self.observers:
            observer.update(*args, **kwargs)
          

# Abstract Class          
class Observer:
    def update(self, *args, **kwargs):
        raise NotImplementedError
      
      
class AmericanStockMarket(Observer):
    def update(self, *args, **kwargs):
        print("American stock market received: {0}\n{1}".format(args, kwargs))
 
 
class EuropeanStockMarket(Observer):
    def update(self, *args, **kwargs):
        print("European stock market received: {0}\n{1}".format(args, kwargs))
 
 
if __name__ == "__main__":
    observable = Observable()
 
    american_observer = AmericanStockMarket()
    observable.register(american_observer)
 
    european_observer = EuropeanStockMarket()
    observable.register(european_observer)
 
    observable.update_observers(event='Market Rally', warning='Short')

American stock market received: ()
{'event': 'Market Rally', 'warning': 'Short'}
European stock market received: ()
{'event': 'Market Rally', 'warning': 'Short'}


### Visitor
The Visitor pattern separates an algorithm from an object structure on which it operates.
<br>

__Implementation__: The objects of the primary type simply *accept* the visitor, then call the visitor’s dynamically-bound member function:

In [20]:
# The Flower hierarchy cannot be changed:
class Flower(object):
    def accept(self, visitor):
        visitor.visit(self)
    def pollinate(self, pollinator):
        print(self, "pollinated by", pollinator)
    def eat(self, eater):
        print(self, "eaten by", eater)
    def __str__(self):
        return self.__class__.__name__

      
class Poppy(Flower):
    pass


class Visitor:
    def __str__(self):
        return self.__class__.__name__

class Bug(Visitor): pass

class Pollinator(Bug): pass

class Predator(Bug): pass


# Add the ability to do "Bee" activities:
class Bee(Pollinator):
    def visit(self, flower):
        flower.pollinate(self)

# Add the ability to do "Fly" activities:
class Fly(Pollinator):
    def visit(self, flower):
        flower.pollinate(self)

# Add the ability to do "Worm" activities:
class Worm(Predator):
    def visit(self, flower):
        flower.eat(self)


flower = Poppy()
flower.accept(Bee())
flower.accept(Fly())
flower.accept(Worm())

Poppy pollinated by Bee
Poppy pollinated by Fly
Poppy eaten by Worm


### Memento
The Memento pattern provides the ability to restore an object to its previous state.
<br>

__Implementation__: Create another object storing the state of this object. Use that object to restore the state when needed.

In [4]:
class Memento(object):
    def __init__(self, state):
        self.__state = state

    def get_saved_state(self):
        return self.__state


class Originator(object):
    # Dummy state variable
    __state = ""

    def set_state(self, state):
        # Set state
        self.__state = state

    def save_to_memento(self):
        # Save state to Memento
        return Memento(self.__state)

    def restore_from_memento(self, memento):
        # Restore state using Memento
        self.__state = memento.get_saved_state()
        
    def get_state(self):
        return self.__state



# list of Memento objects
saved_states = []

originator = Originator()
originator.set_state("State1")
originator.set_state("State2")
saved_states.append(originator.save_to_memento())

originator.set_state("State3")
saved_states.append(originator.save_to_memento())

originator.set_state("State4")

# restore the state
originator.restore_from_memento(saved_states[0])
originator.get_state()

'State2'

### Iterator
The Iterator pattern decouples an algorithm from the specific type of container that it might currently be working with.
<br>

__Implementation__:

In [0]:
class TypedIterator(Iterator):
    def __init__(self, it, this_type):
        self.imp = it
        self.type = this_type

    def hasNext(self):
        return self.imp.hasNext()

    def remove(self): 
        self.imp.remove()
        
    def next(self):
        obj = self.imp.next()
        if not self.type.isInstance(obj):
            raise TypeError("TypedIterator for type " + self.type + \
                            " encountered type: " + obj.getClass())
        return obj

__Note__: Python prescribes a syntax for iterators as part of the language itself, so that language keywords such as `for` work with what Python calls sequences. A sequence has an `__iter__()` method that returns an iterator object. The "iterator protocol" requires `__next__()` return the next element or raise a StopIteration exception upon reaching the end of the sequence. Iterators also provide an `__iter__()` method returning themselves so that they can also be iterated over e.g., using a for loop.